In [1]:
# Import libraries.
import pandas as pd
import numpy as np

import warnings 
warnings.filterwarnings('ignore')

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [2]:
df = pd.read_csv('airbnb_la.csv')
pd.set_option("display.max_columns", None)
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,16228948,https://www.airbnb.com/rooms/16228948,20170502172350,2017-05-03,Robert Ranch in Los Angeles for filming/wedding,Stunning Ranch Estate situated on 20 acres and...,NaN,Stunning Ranch Estate situated on 20 acres and...,none,NaN,NaN,NaN,NaN,NaN,- No noise/music outside allowed after 10 pm. ...,NaN,NaN,https://a0.muscache.com/im/pictures/dd850460-4...,NaN,54345288,https://www.airbnb.com/users/show/54345288,Robert,2016-01-12,"Los Angeles, California, United States",Thank You for considering my house for your ne...,a few days or more,25%,NaN,f,https://a0.muscache.com/im/pictures/1977b9db-1...,https://a0.muscache.com/im/pictures/1977b9db-1...,NaN,1.0,1.0,"['email', 'phone', 'reviews']",t,f,"Acton, CA 93510, United States",NaN,Acton,NaN,Acton,CA,93510,Other (Domestic),"Acton, CA",US,United States,34.530350,-118.227088,t,House,Entire home/apt,10,7.0,5.0,5.0,Real Bed,"{""Wireless Internet"",""Air conditioning"",Pool,K...",NaN,"$3,000.00",NaN,NaN,"$2,000.00",$200.00,1,$0.00,1,1125,5 months ago,NaN,27,53,82,352,2017-05-03,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,t,strict,f,f,1,NaN
1,8909380,https://www.airbnb.com/rooms/8909380,20170502172350,2017-05-03,Nice quiet private ranch house,NaN,Very private and quite . Hiking horse riding a...,Very private and quite . Hiking horse riding a...,none,I'm in the country what's not to love about that,NaN,The train station is only one mile away,Whole house,As much or as little as they want,NaN,https://a0.muscache.com/im/pictures/38165109-1...,https://a0.muscache.com/im/pictures/38165109-1...,https://a0.muscache.com/im/pictures/38165109-1...,https://a0.muscache.com/im/pictures/38165109-1...,44800067,https://www.airbnb.com/users/show/44800067,Kevin,2015-09-22,"Acton, California, United States",NaN,within an hour,100%,NaN,t,https://a0.muscache.com/im/pictures/287ea76c-1...,https://a0.muscache.com/im/pictures/287ea76c-1...,NaN,1.0,1.0,"['email', 'phone', 'reviews']",t,f,"Palmdale, CA 93550, United States",NaN,Acton,NaN,Palmdale,CA,93550,Other (Domestic),"Palmdale, CA",US,United States,34.485431,-118.125380,f,House,Private room,2,1.0,1.0,1.0,Real Bed,{},NaN,$50.00,NaN,NaN,NaN,NaN,1,$0.00,1,1125,2 months ago,NaN,29,45,75,350,2017-05-03,33,2015-12-04,2017-04-22,93.0,10.0,10.0,10.0,10.0,9.0,9.0,f,NaN,NaN,f,flexible,f,f,1,1.91
2,14078522,https://www.airbnb.com/rooms/14078522,20170502172350,2017-05-03,Room w/private bath on Small Ranch,My place is close to Six Flags Magic Mountain ...,Room with queen bed and private bath,My place is close to Six Flags Magic Mountain ...,none,Horse area. Trails for hiking

In [3]:
# Total number of rows and columns.
df.shape

(31253, 95)

In [4]:
# Nulls in each column.
var = df.isnull().sum()

# Nulls are more than half of the rows.
var[var > 15000]

notes                           17438
host_acceptance_rate            31253
neighbourhood_group_cleansed    31253
square_feet                     30869
weekly_price                    23823
monthly_price                   24620
has_availability                31253
license                         31206
dtype: int64

In [5]:
# Drop columns.
df = df.drop(['notes','host_acceptance_rate', 'neighbourhood_group_cleansed', 'square_feet',
              'has_availability', 'license','weekly_price', 'monthly_price'], axis=1)

In [6]:
df = df[['name', 'summary', 'space', 'description', 'neighborhood_overview', 'transit', 
        'room_type', 'amenities', 'city', 'street', 'price', 'bedrooms', 'review_scores_rating']]
df.head()

,name,summary,space,description,neighborhood_overview,transit,room_type,amenities,city,street,price,bedrooms,review_scores_rating
0,Robert Ranch in Los Angeles for filming/wedding,Stunning Ranch Estate situated on 20 acres and...,NaN,Stunning Ranch Estate situated on 20 acres and...,NaN,NaN,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Pool,K...",Acton,"Acton, CA 93510, United States","$3,000.00",5.0,NaN
1,Nice quiet private ranch house,NaN,Very private and quite . Hiking horse riding a...,Very private and quite . Hiking horse riding a...,I'm in the country what's not to love about that,The train station is only one mile away,Private room,{},Palmdale,"Palmdale, CA 93550, United States",$50.00,1.0,93.0
2,Room w/private bath on Small Ranch,My place is close to Six Flags Magic Mountain ...,Room with queen bed and private bath,My place is close to Six Flags Magic Mountain ...,Horse area. Trails for hiking and riding. Quie...,Just 4 miles from Metro commuter Train station,Private room,"{""Wireless Internet"",""Air conditioning"",""Wheel...",Acton,"Acton, CA 93510, United States",$55.00,1.0,100.0
3,Miller Ranch Bunkhouse,"Our place is close to great views, equestrian ...",Please know that we live on a dirt road but it...,"Our place is close to great views, equestrian ...",NaN,NaN,Entire home/apt,"{TV,""Wireless Internet"",""Air conditioning"",Poo...",Acton,"Acton, CA 93510, United States",$150.00,1.0,100.0
4,Studio with full bathroom,Fully furnished studio only a 10 min drive fro...,NaN,Fully furnished studio only a 10 min drive fro...,NaN,NaN,Private room,"{Internet,""Wireless Internet"",Kitchen,""Free pa...",Los Angeles,"West Adams, Los Angeles, CA 90007, United States",$30.00,1.0,93.0


In [7]:
df.isnull().sum()

name                         7
summary                    836
space                     9387
description                 15
neighborhood_overview    12678
transit                  13089
room_type                    0
amenities                    0
city                        11
street                       0
price                        0
bedrooms                    35
review_scores_rating      7527
dtype: int64

In [8]:
df = df.dropna()
df.shape

(13361, 13)

In [9]:
# Combine text columns.
df['Summary'] = df[['summary', 'space', 'description', 'neighborhood_overview', 'transit', 'amenities']].agg(' '.join, axis=1) 

In [10]:
new_df = df.drop(['summary', 'space', 'description', 'neighborhood_overview', 'transit', 'amenities'], axis=1)

In [11]:
# new_df.info()

In [12]:
# Convert price to numeric.
new_df['Price'] = new_df.price.map(lambda x: int(x.replace(x[0], '').replace('.00', '').replace(',', '')))
new_df['Rating'] = new_df['review_scores_rating']

In [13]:
hotel = new_df.drop(['price', 'review_scores_rating'], axis=1)

In [14]:
hotel.head()

,name,room_type,city,street,bedrooms,Summary,Price,Rating
2,Room w/private bath on Small Ranch,Private room,Acton,"Acton, CA 93510, United States",1.0,My place is close to Six Flags Magic Mountain ...,55,100.0
6,The Library,Private room,Los Angeles,"West Adams, Los Angeles, CA 90007, United States",1.0,"My Home is Close to USC, which means you could...",45,98.0
9,Private Room in Beautiful 2BR House,Private room,Los Angeles,"West Adams, Los Angeles, CA 90018, United States",1.0,This beautiful town home is safe and secure in...,80,99.0
15,URBAN Oasis by DTLA/USC. RM 1,Private room,Los Angeles,"Los Angeles, CA 90018, United States",1.0,This modern spacious oasis in the heart of LA ...,55,96.0
16,Room in Historic West Adams District,Private room,Los Angeles,"West Adams, Los Angeles, CA 90007, United States",1.0,This is a personal and cultural space that aim...,80,100.0


In [15]:
hotel.columns

Index(['name', 'room_type', 'city', 'street', 'bedrooms', 'Summary', 'Price',
       'Rating'],
      dtype='object')

In [16]:
hotel.describe()

,bedrooms,Price,Rating
count,13361.000000,13361.000000,13361.000000
mean,1.305965,145.977547,94.816556
std,0.922653,182.371765,6.769546
min,0.000000,10.000000,20.000000
25%,1.000000,70.000000,93.000000
50%,1.000000,100.000000,97.000000
75%,2.000000,160.000000,100.000000
max,10.000000,7500.000000,100.000000


### Make Recommendations

In [17]:
def requirementbased(city, min_price, max_price, min_rating, max_rating, features):
    hotel['city'] = hotel['city'].str.lower()
    hotel['Summary'] = hotel['Summary'].str.lower()
    
    features = features.lower()
    features_tokens = word_tokenize(features)  
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
        
    reqbased = hotel[hotel['city']==city.lower()]
    reqbased = reqbased[(reqbased['Rating'] >= min_rating) & (reqbased['Rating'] <= max_rating)]
    reqbased = reqbased[(reqbased['Price'] >= min_price) & (reqbased['Price'] <= max_price)]

    reqbased = reqbased.set_index(np.arange(reqbased.shape[0]))
    cos=[]

    for i in range(reqbased.shape[0]):
        temp_tokens = word_tokenize(reqbased['Summary'][i])
        temp1_set = {w for w in temp_tokens if not w in sw}
        temp_set = set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        cos.append(len(rvector))
    reqbased['similarity'] = cos
    reqbased = reqbased.sort_values(by='similarity',ascending=False)
    #reqbased.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    return reqbased[['name', 'room_type', 'street', 'bedrooms',
                     'Rating', 'Summary', 'Price', 'similarity']].head(10)

In [18]:
# Prompt the user to enter criteria.
city = input("What is the city you would like to visit?")

min_price = float(input("What is the minimum price you would like for your hotel? ($10 - $7500)"))
max_price = float(input("What is the maximum price you would like for your hotel? ($10 - $7500)"))

min_rating = float(input("What is the minimum rating you would like for your hotel? (20 - 100)"))
max_rating = float(input("What is the maximum rating you would like for your hotel? (20 - 100)"))

features = input("What are the feature you would like?")

What is the city you would like to visit? los angeles
What is the minimum price you would like for your hotel? ($10 - $7500) 20
What is the maximum price you would like for your hotel? ($10 - $7500) 6000
What is the minimum rating you would like for your hotel? (20 - 100) 20
What is the maximum rating you would like for your hotel? (20 - 100) 100
What is the feature you would like? beautiful, personal


In [19]:
requirementbased(city, min_price, max_price, min_rating, max_rating, features)

,name,room_type,street,bedrooms,Rating,Summary,Price
4646,Gazebo Nook in Groovy Artist Loft,Shared room,"Lincoln Heights, Los Angeles, CA 90031, United...",1.0,99.0,beautiful & unique loft in historic art colony...,36
4289,Lrg Single Room in House-Koreatown,Private room,"Mid-Wilshire, Los Angeles, CA 90004, United St...",1.0,99.0,our house is occupied by 3 people and we welco...,45
4902,Midcity & close to everything; I speak some Fr...,Private room,"Los Angeles, CA 90019, United States",1.0,90.0,beautiful sunny bedroom in my 2 brm home. new ...,44
7096,Modern Stylish Private Guesthouse,Entire home/apt,"North Hollywood, Los Angeles, CA 91605, United...",0.0,100.0,less than 10 minutes from universal studios an...,70
6186,Garden House,Private room,"Valley Glen, Los Angeles, CA 91405, United States",1.0,80.0,"private room, private bath with shower. egypti...",85
4913,Salvatore Room @ Cinema Paradiso,Private room,"Mid-City, Los Angeles, CA 90019, United States",1.0,91.0,**** now with air conditioning in the room ***...,55
1779,"Designer Craftsman w/Yard, Deck, Patios & Parking",Entire home/apt,"Los Angeles, CA 90038, United States",3.0,96.0,newly designed mid-century modern craftsman ho...,289
4925,Cinema Paradiso,Entire home/apt,"Mid-City, Los Angeles, CA 90019, United States",3.0,100.0,**** now with air conditioning in two room ***...,179
4715,"Cozy, gorgeous home near Venice",Entire home/apt,"Mar Vista, Los Angeles, CA 90066, United States",3.0,96.0,"we have a cozy, beautiful home in a quiet cul ...",400
6053,"Spacious, private room and bath, Queen sized bed",Private room,"Los Angeles, CA 90025, United States",1.0,97.0,our downstairs room hosts 4 guests. two on the...,75
